In [1]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import Diagonal

n_qubits = 3          
N = 2**n_qubits
L = 1.0              
D = 1.0              
t = 1.0               
x = 1.0               

phi = x * D * t * (2 * np.pi / L)**2

j = np.arange(N) #from 0 to N-1

k_squared = np.where(j < N/2, j**2, (j - N)**2)

unitary_diag = np.exp(-1j * phi * k_squared)

qc = QuantumCircuit(n_qubits)

evolution_gate = Diagonal(unitary_diag)

qc.append(evolution_gate, range(n_qubits))

print("Diagonal Operator Entries:")
print(np.round(unitary_diag, 3))

print("\nCircuit Diagram:")
print(qc)

Diagonal Operator Entries:
[ 1.   +0.j    -0.207-0.978j  0.672-0.741j -0.954+0.301j -0.981+0.193j
 -0.954+0.301j  0.672-0.741j -0.207-0.978j]

Circuit Diagram:
     ┌───────────┐
q_0: ┤0          ├
     │           │
q_1: ┤1 Diagonal ├
     │           │
q_2: ┤2          ├
     └───────────┘


/tmp/ipykernel_84006/2677257778.py:22: DeprecationWarning: The class ``qiskit.circuit.library.generalized_gates.diagonal.Diagonal`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. Use DiagonalGate instead.
  evolution_gate = Diagonal(unitary_diag)
